In [1]:
#Provide the Subscription ID of your existing Azure subscription
subscription_id = "" # <- needs to be the subscription with the Quick-Starts resource group

#Provide values for the existing Resource Group 
resource_group = "" # <- replace XXXXX with your unique identifier

#Provide the Workspace Name and Azure Region of the Azure Machine Learning Workspace
workspace_name = "" # <- replace XXXXX with your unique identifier (should be lowercase)
workspace_region = "" # <- region of your Quick-Starts resource group

aml_compute_target = "cpucluster1" # <- the name of your GPU-enabled cluster

experiment_name = 'r-pipeline'

# project folder for the script files
project_folder = 'src'
src_dir = 'src'

In [2]:
from azureml.core import Workspace
from azureml.core.authentication import InteractiveLoginAuthentication

interactive_auth = InteractiveLoginAuthentication(tenant_id="72f988bf-86f1-41af-91ab-2d7cd011db47")
#subscription_id = '<sub-id>'
#resource_group  = '<rg>'
#workspace_name  = '<workspace>'

try:
    ws = Workspace(subscription_id = subscription_id, resource_group = resource_group, workspace_name = workspace_name, auth = interactive_auth)
    ws.write_config()
    print('Library configuration succeeded')
except:
    print('Workspace not found')

Failure while loading azureml_run_type_providers. Failed to load entrypoint automl = azureml.train.automl.run:AutoMLRun._from_run_dto with exception (numpy 1.20.2 (c:\users\anildwa\miniconda3\envs\aml\lib\site-packages), Requirement.parse('numpy<=1.19.3; sys_platform == "win32"'), {'azureml-dataset-runtime'}).
If you run your code in unattended mode, i.e., where you can't give a user input, then we recommend to use ServicePrincipalAuthentication or MsiAuthentication.
Please refer to aka.ms/aml-notebook-auth for different authentication mechanisms in azureml-sdk.


Library configuration succeeded


In [3]:
# By using the exist_ok param, if the worskpace already exists you get a reference to the existing workspace
# allowing you to re-run this cell multiple times as desired (which is fairly common in notebooks).
ws = Workspace.create(
    name = workspace_name,
    subscription_id = subscription_id,
    resource_group = resource_group, 
    location = workspace_region,
    exist_ok = True)

ws.write_config()
print('Workspace configuration succeeded')

Workspace configuration succeeded


In [4]:
from azureml.core.compute_target import ComputeTargetException
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget

try:
    aml_compute = AmlCompute(ws, aml_compute_target)
    print("found existing compute target.")
except ComputeTargetException:
    print("creating new compute target")
    
    provisioning_config = AmlCompute.provisioning_configuration(vm_size = "STANDARD_DS2_V2",
                                                                min_nodes = 0, 
                                                                max_nodes = 1)    
    aml_compute = ComputeTarget.create(ws, aml_compute_target, provisioning_config)
    aml_compute.wait_for_completion(show_output=True, min_node_count=None, timeout_in_minutes=20)
    
print("Aml Compute attached")

found existing compute target.
Aml Compute attached


In [5]:
from azureml.core import Environment
import os

env = Environment.from_dockerfile(name='r_env', dockerfile=os.path.join("./", 'Dockerfile'))

In [6]:
from azureml.core import ScriptRunConfig

script_config = ScriptRunConfig(source_directory=src_dir,
                            command=['Rscript adlsgen2.R ', '', ''],
                            compute_target=aml_compute,
                            environment=env)

In [7]:
from azureml.pipeline.steps import CommandStep

rstep = CommandStep(name='r_train_step', runconfig=script_config)

In [8]:
from azureml.pipeline.core import Pipeline
from azureml.core import Experiment

pipeline = Pipeline(workspace=ws, steps=[rstep])
pipeline_run = Experiment(ws, 'r-commandstep-pipeline').submit(pipeline)

Created step r_train_step [7fc0d69f][d9d29cbe-7b2b-4b20-86d8-52e7582f337f], (This step will run and generate new outputs)
Submitted PipelineRun e193b7e0-1b5d-4fb5-9bce-966809b29a2d
Link to Azure Machine Learning Portal: https://ml.azure.com/runs/e193b7e0-1b5d-4fb5-9bce-966809b29a2d?wsid=/subscriptions/3e0e14b3-7e28-4da7-97de-0f5cb324f030/resourcegroups/ml/workspaces/ml-service&tid=72f988bf-86f1-41af-91ab-2d7cd011db47
